In [ ]:
# Stage 5 | Notebook 43: 選項平衡規則系統
# Goals: 實作遊戲選項的風險/獎勵平衡機制與技能檢定系統

# Cell1:  Shared Cache Bootstrap
import os, pathlib, torch
import sys
from datetime import datetime

# Shared cache configuration (複製到每本 notebook)
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "../ai_warehouse/cache")

for k, v in {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)
print("[Cache]", AI_CACHE_ROOT, "| GPU:", torch.cuda.is_available())

In [ ]:
# Cell 2: Choice Balancing Theory & Design Principles
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional
from enum import Enum
import random
import json
import math


class ChoiceType(Enum):
    SAFE = "safe"  # Low risk, low reward
    AGGRESSIVE = "aggressive"  # High risk, high reward
    BALANCED = "balanced"  # Medium risk, medium reward
    CREATIVE = "creative"  # Unique approach, variable outcome


class DifficultyLevel(Enum):
    TRIVIAL = 1  # 95% success rate
    EASY = 2  # 80% success rate
    MODERATE = 3  # 65% success rate
    HARD = 4  # 45% success rate
    EXTREME = 5  # 25% success rate


@dataclass
class ChoiceOutcome:
    """Represents the potential outcome of a choice"""

    hp_change: int = 0
    items_gained: List[str] = None
    items_lost: List[str] = None
    flags_set: Dict[str, bool] = None
    story_text: str = ""
    success_probability: float = 0.5

    def __post_init__(self):
        if self.items_gained is None:
            self.items_gained = []
        if self.items_lost is None:
            self.items_lost = []
        if self.flags_set is None:
            self.flags_set = {}


@dataclass
class Choice:
    """A single choice option with balanced risk/reward"""

    id: str
    text: str
    choice_type: ChoiceType
    difficulty: DifficultyLevel
    required_skills: Dict[str, int] = None  # skill_name: minimum_level
    success_outcome: ChoiceOutcome = None
    failure_outcome: ChoiceOutcome = None

    def __post_init__(self):
        if self.required_skills is None:
            self.required_skills = {}
        if self.success_outcome is None:
            self.success_outcome = ChoiceOutcome()
        if self.failure_outcome is None:
            self.failure_outcome = ChoiceOutcome()


print("✓ Choice balancing framework initialized")

In [ ]:
# Cell 3: Skill Check System Core
class SkillCheckSystem:
    """Core skill check and dice rolling system"""

    def __init__(self, base_dice: int = 20, random_seed: Optional[int] = None):
        self.base_dice = base_dice
        self.rng = random.Random(random_seed)

    def calculate_success_rate(
        self,
        player_skill: int,
        difficulty: DifficultyLevel,
        modifiers: Dict[str, int] = None,
    ) -> float:
        """Calculate success probability based on skill vs difficulty"""
        if modifiers is None:
            modifiers = {}

        # Base difficulty thresholds
        difficulty_thresholds = {
            DifficultyLevel.TRIVIAL: 3,
            DifficultyLevel.EASY: 6,
            DifficultyLevel.MODERATE: 10,
            DifficultyLevel.HARD: 14,
            DifficultyLevel.EXTREME: 18,
        }

        threshold = difficulty_thresholds[difficulty]
        total_modifier = player_skill + sum(modifiers.values())

        # Calculate probability of rolling >= (threshold - total_modifier)
        target = max(1, threshold - total_modifier)
        success_rate = max(
            0.05, min(0.95, (self.base_dice - target + 1) / self.base_dice)
        )

        return success_rate

    def perform_check(
        self,
        player_skill: int,
        difficulty: DifficultyLevel,
        modifiers: Dict[str, int] = None,
    ) -> Tuple[bool, int, float]:
        """Perform actual skill check with dice roll"""
        if modifiers is None:
            modifiers = {}

        success_rate = self.calculate_success_rate(player_skill, difficulty, modifiers)
        roll = self.rng.randint(1, self.base_dice)

        # Determine if check succeeds
        threshold_needed = int((1 - success_rate) * self.base_dice) + 1
        success = roll >= threshold_needed

        return success, roll, success_rate

    def get_difficulty_description(self, success_rate: float) -> str:
        """Get human-readable difficulty description"""
        if success_rate >= 0.8:
            return "很容易"
        elif success_rate >= 0.6:
            return "中等難度"
        elif success_rate >= 0.4:
            return "困難"
        else:
            return "極度困難"


# Example usage
skill_system = SkillCheckSystem(random_seed=42)
success, roll, rate = skill_system.perform_check(
    player_skill=8,
    difficulty=DifficultyLevel.MODERATE,
    modifiers={"equipment_bonus": 2},
)
print(
    f"技能檢定: 骰出 {roll}, 成功率 {rate:.2%}, 結果: {'成功' if success else '失敗'}"
)

In [ ]:
# Cell 4: Choice Generator Implementation
class ChoiceGenerator:
    """Generates balanced choices based on game state and context"""

    def __init__(self, skill_system: SkillCheckSystem):
        self.skill_system = skill_system

    def generate_choice_set(
        self, context: Dict, player_state: Dict, num_choices: int = 3
    ) -> List[Choice]:
        """Generate a balanced set of choices for current situation"""
        choices = []

        # Ensure we have different choice types
        choice_types = [ChoiceType.SAFE, ChoiceType.BALANCED, ChoiceType.AGGRESSIVE]
        if num_choices > 3:
            choice_types.extend([ChoiceType.CREATIVE] * (num_choices - 3))

        for i, choice_type in enumerate(choice_types[:num_choices]):
            choice = self._generate_single_choice(
                choice_id=f"choice_{i+1}",
                choice_type=choice_type,
                context=context,
                player_state=player_state,
            )
            choices.append(choice)

        return choices

    def _generate_single_choice(
        self, choice_id: str, choice_type: ChoiceType, context: Dict, player_state: Dict
    ) -> Choice:
        """Generate a single balanced choice"""

        # Determine difficulty based on choice type and player level
        player_level = player_state.get("level", 1)

        difficulty_map = {
            ChoiceType.SAFE: DifficultyLevel.EASY,
            ChoiceType.BALANCED: DifficultyLevel.MODERATE,
            ChoiceType.AGGRESSIVE: DifficultyLevel.HARD,
            ChoiceType.CREATIVE: DifficultyLevel.MODERATE,
        }

        difficulty = difficulty_map[choice_type]

        # Adjust difficulty based on player level
        if player_level <= 2:
            difficulty = DifficultyLevel.EASY
        elif player_level >= 8:
            if difficulty == DifficultyLevel.EASY:
                difficulty = DifficultyLevel.MODERATE

        # Generate choice text and outcomes based on type
        choice_templates = self._get_choice_templates(choice_type, context)
        template = random.choice(choice_templates)

        # Create success and failure outcomes
        success_outcome = self._create_outcome(choice_type, True, player_state)
        failure_outcome = self._create_outcome(choice_type, False, player_state)

        return Choice(
            id=choice_id,
            text=template["text"],
            choice_type=choice_type,
            difficulty=difficulty,
            required_skills=template.get("required_skills", {}),
            success_outcome=success_outcome,
            failure_outcome=failure_outcome,
        )

    def _get_choice_templates(
        self, choice_type: ChoiceType, context: Dict
    ) -> List[Dict]:
        """Get choice text templates based on type and context"""
        location = context.get("location", "unknown")

        templates = {
            ChoiceType.SAFE: [
                {
                    "text": f"小心地探索{location}，避免不必要的風險",
                    "required_skills": {"perception": 3},
                },
                {
                    "text": "等待並觀察周圍環境的變化",
                    "required_skills": {"patience": 2},
                },
                {"text": "尋找其他較安全的路線", "required_skills": {"navigation": 4}},
            ],
            ChoiceType.BALANCED: [
                {
                    "text": f"直接穿越{location}，保持警戒",
                    "required_skills": {"combat": 5},
                },
                {
                    "text": "嘗試與當地人交談獲取信息",
                    "required_skills": {"charisma": 6},
                },
                {
                    "text": "使用你的技能解決當前問題",
                    "required_skills": {"intelligence": 5},
                },
            ],
            ChoiceType.AGGRESSIVE: [
                {
                    "text": f"強行突破{location}的阻礙",
                    "required_skills": {"strength": 8},
                },
                {"text": "發動先制攻擊", "required_skills": {"combat": 10}},
                {"text": "冒險嘗試最直接的方法", "required_skills": {"courage": 7}},
            ],
            ChoiceType.CREATIVE: [
                {
                    "text": "使用你的特殊物品來解決問題",
                    "required_skills": {"creativity": 6},
                },
                {
                    "text": "嘗試一個前所未有的方法",
                    "required_skills": {"innovation": 8},
                },
                {
                    "text": "結合多種技能創造新的解決方案",
                    "required_skills": {"synthesis": 7},
                },
            ],
        }

        return templates.get(choice_type, templates[ChoiceType.BALANCED])

    def _create_outcome(
        self, choice_type: ChoiceType, success: bool, player_state: Dict
    ) -> ChoiceOutcome:
        """Create balanced outcomes based on choice type and success"""

        base_outcomes = {
            ChoiceType.SAFE: {
                True: {
                    "hp_change": 0,
                    "items": ["small_potion"],
                    "text": "你安全地完成了行動",
                },
                False: {
                    "hp_change": -1,
                    "items": [],
                    "text": "雖然謹慎，但仍遇到了小麻煩",
                },
            },
            ChoiceType.BALANCED: {
                True: {
                    "hp_change": 1,
                    "items": ["useful_item"],
                    "text": "你的策略奏效了",
                },
                False: {"hp_change": -2, "items": [], "text": "計劃沒有按預期進行"},
            },
            ChoiceType.AGGRESSIVE: {
                True: {
                    "hp_change": 3,
                    "items": ["rare_item", "bonus_loot"],
                    "text": "大膽的行動帶來了豐厚回報",
                },
                False: {"hp_change": -5, "items": [], "text": "冒險的代價是慘重的"},
            },
            ChoiceType.CREATIVE: {
                True: {
                    "hp_change": 2,
                    "items": ["unique_item"],
                    "text": "創新的方法開闢了新的可能",
                },
                False: {"hp_change": -1, "items": [], "text": "實驗性的嘗試沒有成功"},
            },
        }

        outcome_data = base_outcomes[choice_type][success]

        return ChoiceOutcome(
            hp_change=outcome_data["hp_change"],
            items_gained=outcome_data["items"].copy(),
            story_text=outcome_data["text"],
        )


# Example choice generation
generator = ChoiceGenerator(skill_system)
context = {"location": "古老的圖書館", "threat_level": "medium"}
player_state = {"level": 3, "hp": 8, "skills": {"combat": 6, "intelligence": 8}}

choices = generator.generate_choice_set(context, player_state)
for i, choice in enumerate(choices, 1):
    print(
        f"選項 {i}: {choice.text} ({choice.choice_type.value}, {choice.difficulty.name})"
    )

In [ ]:
# Cell 5: Dynamic Difficulty Adjustment Algorithm
class DynamicDifficultyAdjuster:
    """Adjusts game difficulty based on player performance and progress"""

    def __init__(self, target_success_rate: float = 0.65):
        self.target_success_rate = target_success_rate
        self.performance_history = []
        self.adjustment_factor = 0.1

    def record_performance(self, success: bool, difficulty: DifficultyLevel):
        """Record the outcome of a player's choice"""
        self.performance_history.append(
            {
                "success": success,
                "difficulty": difficulty,
                "timestamp": len(self.performance_history),
            }
        )

        # Keep only recent history (last 10 attempts)
        if len(self.performance_history) > 10:
            self.performance_history.pop(0)

    def get_current_performance_rate(self) -> float:
        """Calculate current success rate from recent history"""
        if not self.performance_history:
            return self.target_success_rate

        recent_successes = sum(
            1 for record in self.performance_history[-5:] if record["success"]
        )
        return recent_successes / min(5, len(self.performance_history))

    def suggest_difficulty_adjustment(
        self, base_difficulty: DifficultyLevel
    ) -> DifficultyLevel:
        """Suggest difficulty adjustment based on performance"""
        current_rate = self.get_current_performance_rate()

        # If player is succeeding too often, increase difficulty
        if current_rate > self.target_success_rate + 0.15:
            return self._increase_difficulty(base_difficulty)
        # If player is failing too often, decrease difficulty
        elif current_rate < self.target_success_rate - 0.15:
            return self._decrease_difficulty(base_difficulty)
        else:
            return base_difficulty

    def _increase_difficulty(self, difficulty: DifficultyLevel) -> DifficultyLevel:
        """Increase difficulty by one level if possible"""
        difficulty_order = list(DifficultyLevel)
        current_index = difficulty_order.index(difficulty)

        if current_index < len(difficulty_order) - 1:
            return difficulty_order[current_index + 1]
        return difficulty

    def _decrease_difficulty(self, difficulty: DifficultyLevel) -> DifficultyLevel:
        """Decrease difficulty by one level if possible"""
        difficulty_order = list(DifficultyLevel)
        current_index = difficulty_order.index(difficulty)

        if current_index > 0:
            return difficulty_order[current_index - 1]
        return difficulty

    def get_adjustment_summary(self) -> Dict:
        """Get summary of current adjustment status"""
        return {
            "current_success_rate": self.get_current_performance_rate(),
            "target_success_rate": self.target_success_rate,
            "total_attempts": len(self.performance_history),
            "recent_trend": (
                "improving"
                if self.get_current_performance_rate() > 0.5
                else "struggling"
            ),
        }


# Example usage
difficulty_adjuster = DynamicDifficultyAdjuster()

# Simulate some gameplay
for i in range(8):
    success = random.choice([True, False])  # Random for demo
    difficulty_adjuster.record_performance(success, DifficultyLevel.MODERATE)

suggested_difficulty = difficulty_adjuster.suggest_difficulty_adjustment(
    DifficultyLevel.MODERATE
)
summary = difficulty_adjuster.get_adjustment_summary()

print("難度調整建議:")
print(f"建議難度: {suggested_difficulty.name}")
print(f"當前成功率: {summary['current_success_rate']:.2%}")
print(f"目標成功率: {summary['target_success_rate']:.2%}")

In [ ]:
# Cell 6: Randomness & Predictability Balance
class RandomnessController:
    """Controls randomness to balance unpredictability with player agency"""

    def __init__(self, base_seed: int = 42):
        self.base_seed = base_seed
        self.sequence_rng = random.Random(base_seed)
        self.choice_rng = random.Random()
        self.chaos_factor = 0.1  # How much pure randomness to inject

    def set_controlled_randomness(
        self, player_actions: List[str], turn_number: int
    ) -> None:
        """Set predictable randomness based on player actions and turn"""
        # Create seed from player actions hash and turn number
        action_hash = hash("".join(player_actions)) % 10000
        seed = self.base_seed + action_hash + turn_number
        self.choice_rng.seed(seed)

    def should_add_chaos_event(self) -> bool:
        """Decide if a chaos event should occur (low probability)"""
        return self.sequence_rng.random() < self.chaos_factor

    def get_predictable_outcome_modifier(self, choice_history: List[str]) -> float:
        """Get outcome modifier based on choice patterns"""
        if len(choice_history) < 3:
            return 1.0

        # Reward varied choices, penalize repetitive ones
        recent_choices = choice_history[-3:]
        unique_choices = len(set(recent_choices))

        if unique_choices == 1:  # All same choices
            return 0.8  # Slight penalty for monotony
        elif unique_choices == 3:  # All different choices
            return 1.2  # Bonus for variety
        else:
            return 1.0  # No modifier

    def generate_story_variation(
        self, base_text: str, variation_level: float = 0.3
    ) -> str:
        """Add controlled variation to story text"""
        variations = {
            "成功": ["成功", "順利完成", "成果豐碩", "表現出色"],
            "失敗": ["失敗", "遭遇挫折", "計劃受阻", "結果不佳"],
            "困難": ["困難", "挑戰", "阻礙", "考驗"],
            "容易": ["容易", "簡單", "輕鬆", "毫不費力"],
        }

        modified_text = base_text
        for original, alternatives in variations.items():
            if original in base_text and self.choice_rng.random() < variation_level:
                replacement = self.choice_rng.choice(alternatives)
                modified_text = modified_text.replace(original, replacement, 1)

        return modified_text


# Example controlled randomness
randomness_controller = RandomnessController()
player_actions = ["explore", "fight", "negotiate"]
randomness_controller.set_controlled_randomness(player_actions, turn_number=5)

chaos_event = randomness_controller.should_add_chaos_event()
outcome_modifier = randomness_controller.get_predictable_outcome_modifier(
    ["aggressive", "balanced", "safe"]
)

print(f"隨機性控制:")
print(f"混沌事件: {'是' if chaos_event else '否'}")
print(f"結果修正值: {outcome_modifier:.1f}")

In [ ]:
# Cell 7: Choice Validation & Testing System
class ChoiceValidator:
    """Validates and tests choice balance and fairness"""

    def __init__(self):
        self.validation_rules = [
            self._validate_risk_reward_balance,
            self._validate_choice_diversity,
            self._validate_skill_requirements,
            self._validate_outcome_fairness,
        ]

    def validate_choice_set(
        self, choices: List[Choice], player_state: Dict
    ) -> Dict[str, any]:
        """Validate a complete set of choices"""
        validation_results = {
            "valid": True,
            "warnings": [],
            "errors": [],
            "balance_score": 0.0,
        }

        for rule in self.validation_rules:
            try:
                rule_result = rule(choices, player_state)
                if rule_result["errors"]:
                    validation_results["errors"].extend(rule_result["errors"])
                    validation_results["valid"] = False
                if rule_result["warnings"]:
                    validation_results["warnings"].extend(rule_result["warnings"])
                validation_results["balance_score"] += rule_result.get("score", 0)
            except Exception as e:
                validation_results["errors"].append(f"Validation rule failed: {e}")
                validation_results["valid"] = False

        # Average the balance score
        validation_results["balance_score"] /= len(self.validation_rules)

        return validation_results

    def _validate_risk_reward_balance(
        self, choices: List[Choice], player_state: Dict
    ) -> Dict:
        """Ensure risk and reward are properly balanced"""
        result = {"errors": [], "warnings": [], "score": 0}

        for choice in choices:
            success_value = self._calculate_outcome_value(choice.success_outcome)
            failure_cost = abs(self._calculate_outcome_value(choice.failure_outcome))

            # High-risk choices should have proportionally high rewards
            if choice.choice_type == ChoiceType.AGGRESSIVE:
                if success_value <= failure_cost:
                    result["warnings"].append(
                        f"High-risk choice '{choice.text}' has insufficient reward"
                    )
                else:
                    result["score"] += 1
            elif choice.choice_type == ChoiceType.SAFE:
                if failure_cost > success_value * 0.5:
                    result["warnings"].append(
                        f"Safe choice '{choice.text}' has excessive risk"
                    )
                else:
                    result["score"] += 1

        result["score"] /= len(choices)
        return result

    def _validate_choice_diversity(
        self, choices: List[Choice], player_state: Dict
    ) -> Dict:
        """Ensure choices offer diverse approaches"""
        result = {"errors": [], "warnings": [], "score": 0}

        choice_types = [choice.choice_type for choice in choices]
        unique_types = len(set(choice_types))

        if unique_types < 2:
            result["errors"].append(
                "Insufficient choice diversity - need different types"
            )
        elif unique_types == len(choices):
            result["score"] = 1.0
        else:
            result["score"] = unique_types / len(choices)

        return result

    def _validate_skill_requirements(
        self, choices: List[Choice], player_state: Dict
    ) -> Dict:
        """Ensure skill requirements are reasonable"""
        result = {"errors": [], "warnings": [], "score": 0}

        player_skills = player_state.get("skills", {})
        accessible_choices = 0

        for choice in choices:
            is_accessible = True
            for skill, required_level in choice.required_skills.items():
                if player_skills.get(skill, 0) < required_level:
                    is_accessible = False
                    break
            if is_accessible:
                accessible_choices += 1

        if accessible_choices == 0:
            result["errors"].append("No choices are accessible to player")
        elif accessible_choices == len(choices):
            result["warnings"].append("All choices too easy - no skill challenges")
            result["score"] = 0.7
        else:
            result["score"] = 1.0

        return result

    def _validate_outcome_fairness(
        self, choices: List[Choice], player_state: Dict
    ) -> Dict:
        """Ensure outcomes are fair and not game-breaking"""
        result = {"errors": [], "warnings": [], "score": 0}

        player_hp = player_state.get("hp", 10)
        fatal_choices = 0

        for choice in choices:
            failure_hp_loss = abs(choice.failure_outcome.hp_change)
            if failure_hp_loss >= player_hp:
                fatal_choices += 1
                result["warnings"].append(
                    f"Choice '{choice.text}' can be instantly fatal"
                )

        if fatal_choices >= len(choices):
            result["errors"].append("All choices can result in instant death")
        else:
            result["score"] = 1.0 - (fatal_choices / len(choices))

        return result

    def _calculate_outcome_value(self, outcome: ChoiceOutcome) -> float:
        """Calculate numeric value of an outcome"""
        value = outcome.hp_change * 2  # HP is worth 2 points each
        value += len(outcome.items_gained) * 3  # Items worth 3 points each
        value -= len(outcome.items_lost) * 3
        return value


# Example validation
validator = ChoiceValidator()
validation_result = validator.validate_choice_set(choices, player_state)

print("選項驗證結果:")
print(f"有效: {'是' if validation_result['valid'] else '否'}")
print(f"平衡分數: {validation_result['balance_score']:.2f}")
if validation_result["warnings"]:
    print("警告:", validation_result["warnings"])

In [ ]:
# Cell 8: RAG Integration for Context-Aware Choices
# Note: This assumes RAG components from Stage 2 are available
class RAGChoiceEnhancer:
    """Enhances choice generation with RAG-retrieved world knowledge"""

    def __init__(self, retriever=None):
        self.retriever = retriever  # From Stage 2 RAG system
        self.world_knowledge_cache = {}

    def enhance_choices_with_lore(
        self, choices: List[Choice], context: Dict
    ) -> List[Choice]:
        """Add world-specific details to choices using RAG"""
        if not self.retriever:
            # Fallback: use mock world knowledge
            return self._enhance_with_mock_lore(choices, context)

        enhanced_choices = []
        location = context.get("location", "")

        # Retrieve relevant world knowledge
        query = f"location {location} history culture dangers opportunities"
        try:
            knowledge_chunks = self.retriever.search(query, k=3)
            world_context = " ".join([chunk[0] for chunk in knowledge_chunks])
        except:
            world_context = "An ancient place with mysterious properties."

        for choice in choices:
            enhanced_choice = self._enhance_single_choice(
                choice, world_context, context
            )
            enhanced_choices.append(enhanced_choice)

        return enhanced_choices

    def _enhance_single_choice(
        self, choice: Choice, world_context: str, situation_context: Dict
    ) -> Choice:
        """Enhance a single choice with world knowledge"""
        # Create enhanced copy
        enhanced = Choice(
            id=choice.id,
            text=choice.text,
            choice_type=choice.choice_type,
            difficulty=choice.difficulty,
            required_skills=choice.required_skills.copy(),
            success_outcome=choice.success_outcome,
            failure_outcome=choice.failure_outcome,
        )

        # Add world-specific flavor to choice text
        location = situation_context.get("location", "")
        if "古老" in world_context and "圖書館" in location:
            if choice.choice_type == ChoiceType.SAFE:
                enhanced.text += "，小心不要觸碰古老的陷阱機關"
            elif choice.choice_type == ChoiceType.AGGRESSIVE:
                enhanced.text += "，無視古老的詛咒傳說"
            elif choice.choice_type == ChoiceType.CREATIVE:
                enhanced.text += "，利用古老的魔法知識"

        # Modify outcomes based on world knowledge
        if "危險" in world_context:
            # Increase risk for all choices in dangerous locations
            enhanced.failure_outcome.hp_change -= 1
        if "寶藏" in world_context:
            # Increase rewards for successful choices
            enhanced.success_outcome.items_gained.append("ancient_artifact")

        return enhanced

    def _enhance_with_mock_lore(
        self, choices: List[Choice], context: Dict
    ) -> List[Choice]:
        """Fallback enhancement using mock world knowledge"""
        location = context.get("location", "unknown")
        mock_lore = {
            "古老的圖書館": "充滿失落知識與隱藏危險的神秘場所",
            "黑暗森林": "野獸出沒，但也有珍貴藥草的危險之地",
            "廢棄城堡": "曾經輝煌，現在充滿幽靈與寶藏",
        }

        lore = mock_lore.get(location, "一個普通的地方")

        enhanced_choices = []
        for choice in choices:
            enhanced_text = f"{choice.text}（{lore}）"
            enhanced_choice = Choice(
                id=choice.id,
                text=enhanced_text,
                choice_type=choice.choice_type,
                difficulty=choice.difficulty,
                required_skills=choice.required_skills,
                success_outcome=choice.success_outcome,
                failure_outcome=choice.failure_outcome,
            )
            enhanced_choices.append(enhanced_choice)

        return enhanced_choices


# Example RAG enhancement (mock)
rag_enhancer = RAGChoiceEnhancer()
enhanced_choices = rag_enhancer.enhance_choices_with_lore(choices, context)

print("RAG 增強選項:")
for i, choice in enumerate(enhanced_choices, 1):
    print(f"  {i}. {choice.text}")

In [ ]:
# Cell 9: Smoke Test - Complete Choice Generation & Skill Check Flow
def smoke_test_choice_balancing():
    """Complete smoke test for choice balancing system"""
    print("=== 選項平衡系統煙霧測試 ===")

    # Initialize systems
    skill_system = SkillCheckSystem(random_seed=42)
    generator = ChoiceGenerator(skill_system)
    validator = ChoiceValidator()
    difficulty_adjuster = DynamicDifficultyAdjuster()
    randomness_controller = RandomnessController()

    # Set up test scenario
    context = {"location": "古老的圖書館", "threat_level": "medium", "chapter": 3}

    player_state = {
        "level": 4,
        "hp": 12,
        "max_hp": 15,
        "skills": {
            "combat": 7,
            "intelligence": 9,
            "perception": 6,
            "charisma": 5,
            "creativity": 8,
        },
        "inventory": ["sword", "healing_potion"],
        "choice_history": ["balanced", "safe", "aggressive"],
    }

    print(f"場景: {context['location']}")
    print(
        f"玩家狀態: HP {player_state['hp']}/{player_state['max_hp']}, 等級 {player_state['level']}"
    )
    print()

    # Generate balanced choices
    choices = generator.generate_choice_set(context, player_state, num_choices=3)

    # Validate choices
    validation = validator.validate_choice_set(choices, player_state)
    print(f"選項驗證: {'通過' if validation['valid'] else '失敗'}")
    print(f"平衡分數: {validation['balance_score']:.2f}/1.0")

    if validation["warnings"]:
        print("警告:", validation["warnings"])
    if validation["errors"]:
        print("錯誤:", validation["errors"])
    print()

    # Display choices with skill check probabilities
    print("可用選項:")
    for i, choice in enumerate(choices, 1):
        # Calculate success probability
        main_skill = (
            list(choice.required_skills.keys())[0]
            if choice.required_skills
            else "combat"
        )
        skill_level = player_state["skills"].get(main_skill, 5)

        success_rate = skill_system.calculate_success_rate(
            skill_level, choice.difficulty, {"equipment": 1}  # Mock equipment bonus
        )

        difficulty_desc = skill_system.get_difficulty_description(success_rate)

        print(f"  {i}. {choice.text}")
        print(f"     類型: {choice.choice_type.value} | 難度: {choice.difficulty.name}")
        print(f"     成功率: {success_rate:.1%} ({difficulty_desc})")

        if choice.required_skills:
            skills_text = ", ".join(
                [f"{skill}≥{level}" for skill, level in choice.required_skills.items()]
            )
            print(f"     技能需求: {skills_text}")

        print(
            f"     成功: {choice.success_outcome.story_text} (HP{choice.success_outcome.hp_change:+d})"
        )
        print(
            f"     失敗: {choice.failure_outcome.story_text} (HP{choice.failure_outcome.hp_change:+d})"
        )
        print()

    # Simulate player choosing option 2 (balanced choice)
    chosen_choice = choices[1]
    print(f"玩家選擇: {chosen_choice.text}")

    # Perform skill check
    main_skill = (
        list(chosen_choice.required_skills.keys())[0]
        if chosen_choice.required_skills
        else "combat"
    )
    skill_level = player_state["skills"].get(main_skill, 5)

    success, roll, success_rate = skill_system.perform_check(
        skill_level, chosen_choice.difficulty, {"equipment": 1}
    )

    print(f"技能檢定: 骰出 {roll}/20 (需要 {int((1-success_rate)*20)+1}+ 成功)")
    print(f"結果: {'成功' if success else '失敗'}")

    # Apply outcome
    outcome = (
        chosen_choice.success_outcome if success else chosen_choice.failure_outcome
    )
    new_hp = player_state["hp"] + outcome.hp_change

    print(f"結果: {outcome.story_text}")
    print(f"HP 變化: {player_state['hp']} → {new_hp} ({outcome.hp_change:+d})")

    if outcome.items_gained:
        print(f"獲得物品: {', '.join(outcome.items_gained)}")

    # Record performance for difficulty adjustment
    difficulty_adjuster.record_performance(success, chosen_choice.difficulty)

    # Get difficulty adjustment suggestion
    suggested_difficulty = difficulty_adjuster.suggest_difficulty_adjustment(
        chosen_choice.difficulty
    )
    adjustment_summary = difficulty_adjuster.get_adjustment_summary()

    print()
    print("難度調整建議:")
    print(f"當前成功率: {adjustment_summary['current_success_rate']:.1%}")
    print(f"建議難度: {suggested_difficulty.name}")

    print("\n✓ 煙霧測試完成 - 所有系統運作正常")

    return {
        "choices_generated": len(choices),
        "validation_passed": validation["valid"],
        "balance_score": validation["balance_score"],
        "skill_check_success": success,
        "difficulty_appropriate": abs(adjustment_summary["current_success_rate"] - 0.65)
        < 0.2,
    }


# Run smoke test
test_results = smoke_test_choice_balancing()

In [ ]:
# Cell 10: Usage Guidelines & Extension Points
print("\n=== 使用時機與擴展方向 ===")

usage_guidelines = """
## 何時使用此選項平衡系統

### 適用場景
1. **文字冒險遊戲**: 為每個場景提供平衡的策略選擇
2. **互動式小說**: 確保讀者的選擇有意義的後果
3. **教育遊戲**: 提供不同難度的挑戰路徑
4. **RPG 對話系統**: 平衡外交、威脅、欺騙等選項

### 核心優勢
- **動態平衡**: 根據玩家表現自動調整難度
- **多樣性保證**: 確保每次都有不同類型的選項
- **技能整合**: 將玩家屬性有意義地融入選擇
- **風險/獎勵透明**: 玩家可以理解每個選擇的潛在後果

### 關鍵參數調整
```python
# 調整目標成功率 (建議 0.6-0.8)
difficulty_adjuster = DynamicDifficultyAdjuster(target_success_rate=0.7)

# 調整混沌因子 (建議 0.05-0.2)
randomness_controller = RandomnessController()
randomness_controller.chaos_factor = 0.15

# 調整骰子類型 (d20, d100等)
skill_system = SkillCheckSystem(base_dice=100)
```

### 擴展方向

1. **情感系統整合**
   - 根據玩家情緒狀態影響選項傾向
   - 追蹤長期關係與聲譽

2. **群體決策**
   - 多玩家協作選擇
   - AI NPC 隊友的建議系統

3. **時間壓力機制**
   - 限時選擇影響可用選項
   - 拖延的後果與機會成本

4. **選項預覽系統**
   - 顯示選擇的可能後果範圍
   - 基於過往經驗的成功率預測

5. **自適應敘事**
   - 根據選擇模式調整故事風格
   - 個人化的冒險體驗

### 效能考量
- 選項生成: O(n) 其中 n = 選項數量
- 技能檢定: O(1) 常數時間
- 驗證系統: O(n) 每個選項一次驗證
- 建議緩存世界知識以提升 RAG 查詢效率
"""

print(usage_guidelines)

# Performance metrics
print("\n效能指標:")
print(f"選項生成數量: {test_results['choices_generated']}")
print(f"驗證通過: {'是' if test_results['validation_passed'] else '否'}")
print(f"平衡分數: {test_results['balance_score']:.2f}/1.0")
print(f"技能檢定成功: {'是' if test_results['skill_check_success'] else '否'}")
print(f"難度適中: {'是' if test_results['difficulty_appropriate'] else '否'}")


# Extension example: Emotion-based choice weighting
class EmotionBasedChoiceWeighting:
    """Example extension: weight choices based on player emotional state"""

    def __init__(self):
        self.emotion_weights = {
            "angry": {"aggressive": 1.5, "safe": 0.5, "balanced": 1.0, "creative": 0.8},
            "cautious": {
                "aggressive": 0.3,
                "safe": 1.8,
                "balanced": 1.2,
                "creative": 1.0,
            },
            "curious": {
                "aggressive": 0.8,
                "safe": 0.6,
                "balanced": 1.0,
                "creative": 1.6,
            },
            "confident": {
                "aggressive": 1.3,
                "safe": 0.7,
                "balanced": 1.1,
                "creative": 1.2,
            },
        }

    def adjust_choice_appeal(
        self, choices: List[Choice], player_emotion: str
    ) -> List[Choice]:
        """Adjust choice appeal based on emotional state"""
        if player_emotion not in self.emotion_weights:
            return choices

        weights = self.emotion_weights[player_emotion]

        for choice in choices:
            weight = weights.get(choice.choice_type.value, 1.0)
            # Could modify success rates, text emphasis, or ordering
            if weight > 1.2:
                choice.text = f"💡 {choice.text}"  # Highlight appealing choices
            elif weight < 0.7:
                choice.text = f"⚠️ {choice.text}"  # Warning for less appealing

        return choices


print("\n擴展範例: 情感導向選項權重系統已實作")
print("💡 表示當前情緒狀態下較有吸引力的選項")
print("⚠️ 表示可能不符合當前情緒的選項")